In [ ]:
from RAG_QA import *

dataset = build_corpus_dataset()

In [ ]:
dataset.push_to_hub('pat-jj/nyt10_corpus')

In [ ]:
dataset

In [ ]:
import faiss
faiss_num_dim = 768
faiss_num_links = 128
index = faiss.IndexHNSWFlat(faiss_num_dim, faiss_num_links, faiss.METRIC_INNER_PRODUCT)
dataset.add_faiss_index("embeddings", custom_index=index)

In [28]:
from RAG_QA import *
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration, DPRContextEncoder, \
    DPRContextEncoderTokenizer

def load_retriever_and_generator_(dataset, load_model=False):
    rag_model_name = "facebook/rag-token-nq"

    if load_model:
        retriever = RagRetriever.from_pretrained("pat-jj/nyt10-finetuned-rag-retriever", index_name="exact")
        model = RagTokenForGeneration.from_pretrained("pat-jj/nyt10-finetuned-rag-generator", index_name="exact")
    else:
        retriever = RagRetriever.from_pretrained(
            rag_model_name, index_name="custom", indexed_dataset=dataset
        )
        # initialize with RagRetriever to do everything in one forward call
        model = RagTokenForGeneration.from_pretrained(rag_model_name, retriever=retriever).to(device)

        # try:
        #     retriever.push_to_hub("pat-jj/nyt10-finetuned-rag-retriever")
        #     print("Successfully push the retriever to the model hub!")
        #     model.push_to_hub("pat-jj/nyt10-finetuned-rag-generator")
        #     print("Successfully push the generator to the model hub!")
        # except:
        #     print("fail to push the retriever/generator to model hub")

    tokenizer = RagTokenizer.from_pretrained(rag_model_name)

    return model, tokenizer, retriever

model, tokenizer, retriever = load_retriever_and_generator_(dataset)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [46]:
questions = [
    "When was Google founded?",
    "Where is Hunan located?",
    "Who founded Microsoft?",
    "Who founded Netflix?",
    "Nationality of Bill Gates?"
]

for q in questions:
    results = ask_question(model, tokenizer, q)
    print(results)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/transformers/generation_utils.py:2169: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


{'answers': [' eight years ago', ' eight years old', ' 2008', ' 8 years ago', ' eight years ago.'], 'titles': ['/business/company/place_founded', '/location/location/contains', '/location/location/contains', '/business/company/founders', '/business/company_shareholder/major_shareholder_of']}
{'answers': [' provinces in east - central china', ' east - central china', ' province in east - central china', ' province in east-central china', ' provinces in east-central china'], 'titles': ['/location/cn_province/capital', '/location/location/contains', '/location/location/contains', '/location/administrative_division/country', '/location/administrative_division/country']}
{'answers': [' bill_gates', ' bill_ gates', ' bill_gate', ' bill_gnates', ' bill_gates'], 'titles': ['/business/company/founders', '/business/person/company', '/business/person/company', '/business/company/founders', '/business/company/founders']}
{'answers': [' sir mr. hastings', 'mr. hastings', ' sir robert hastings', ' s

In [47]:
questions = [
    "What is Microsoft?"
]

for q in questions:
    results = ask_question(model, tokenizer, q)
    print(results)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/transformers/generation_utils.py:2169: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


{'answers': ['', ',', ',', 'pt', ' the'], 'titles': ['/business/person/company', '/business/person/company', '/business/company/founders', '/business/person/company', '/business/person/company']}
